<a href="https://colab.research.google.com/github/yzm9393/swineBRET-ICD/blob/main/03_Train_Teacher_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Train the 11 teacher models without calibration (Code not used)

In [ ]:
# import pandas as pd
# import numpy as np
# import joblib
# import os
# import re
# import ast
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.utils import resample

# # --- 1. Configuration ---
# ANNOTATED_DATA_FILE = '/Users/zimoyang/Documents/swine_project/data/training_set_processed.csv'
# TEXT_COLUMN = 'anonymized_text_for_ml' # Use your preprocessed/anonymized text column
# LABEL_VECTOR_COLUMN = 'expert_labels_vector'
# RANDOM_STATE = 42
# MODEL_OUTPUT_DIR = '/Users/zimoyang/Documents/swine_project/teacher_models'

# # The final, ordered list of 11 "modeling" labels.
# # The order of this list MUST match the order used to create the vector.
# FINAL_MODELING_LABELS = [
#     '[01] Certain infectious or parasitic diseases',
#     '[08] Diseases of the nervous system',
#     '[12] Diseases of the respiratory system',
#     '[13] Diseases of the digestive system',
#     '[14] Diseases of the skin',
#     '[15] Diseases of the musculoskeletal system or connective tissue',
#     '[18] Pregnancy, childbirth or the puerperium',
#     '[19] Certain conditions originating in the perinatal period',
#     'Monitoring',
#     'Unknown',
#     'Symptoms not classified elsewhere'
# ]

# # --- 2. Load Gold Standard Data ---
# print(f"--- Loading Gold Standard Annotated Data from '{ANNOTATED_DATA_FILE}' ---")
# df_master = pd.read_csv(ANNOTATED_DATA_FILE)

# # Isolate the 2000 records annotated by Dr. Poljak
# df_gold = df_master.dropna(subset=[LABEL_VECTOR_COLUMN]).copy()

# # The 'expert_labels_vector' is loaded as a string, so we need to safely convert it back to a list
# df_gold[LABEL_VECTOR_COLUMN] = df_gold[LABEL_VECTOR_COLUMN].apply(ast.literal_eval)
# print(f"Loaded {len(df_gold)} gold-standard records.")

# # Create the output directory if it doesn't exist
# os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# # --- 3. Loop Through Each Label to Train and Save a Specialist Model ---
# # We use enumerate to get both the index (i) and the name of the label
# for label_index, label_name in enumerate(FINAL_MODELING_LABELS):
#     print("\n" + "="*80)
#     print(f"--- Training Binary Teacher Model for: {label_name} (Index: {label_index}) ---")
#     print("="*80)

#     # --- THIS IS THE KEY CHANGE ---
#     # Create a specific binary target (y) for this label by extracting the
#     # value at the correct index from each record's vector.
#     y = np.array([vector[label_index] for vector in df_gold[LABEL_VECTOR_COLUMN]])

#     # Add this temporary column to the dataframe for easy filtering/sampling
#     df_gold['current_target'] = y

#     # Check for positive samples.
#     if df_gold['current_target'].sum() == 0:
#         print(f"Skipping '{label_name}' as there are no positive examples.")
#         continue

#     # b. Address Class Imbalance by Downsampling the Negative Class
#     positive_class = df_gold[df_gold['current_target'] == 1]
#     negative_class = df_gold[df_gold['current_target'] == 0]

#     negative_downsampled = resample(negative_class,
#                                     replace=False,
#                                     n_samples=len(positive_class),
#                                     random_state=RANDOM_STATE)

#     balanced_training_data = pd.concat([positive_class, negative_downsampled])

#     print(f"Created a balanced training set with {len(balanced_training_data)} records.")

#     X_train = balanced_training_data[TEXT_COLUMN] # Use the correct text column
#     y_train = balanced_training_data['current_target']

#     # c. Create and Fit a New TF-IDF Vectorizer for this model
#     vectorizer = TfidfVectorizer(max_features=3000, stop_words='english')
#     X_train_tfidf = vectorizer.fit_transform(X_train)

#     # d. Train a Simple Classifier Model
#     model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
#     model.fit(X_train_tfidf, y_train)

#     # e. Save the Trained Model AND its specific Vectorizer
#     safe_label_name = re.sub(r'[\W_]+', '', label_name)
#     model_path = os.path.join(MODEL_OUTPUT_DIR, f'model_{safe_label_name}.joblib')
#     vectorizer_path = os.path.join(MODEL_OUTPUT_DIR, f'vectorizer_{safe_label_name}.joblib')

#     joblib.dump(model, model_path)
#     joblib.dump(vectorizer, vectorizer_path)
#     print(f"Saved model and vectorizer for '{label_name}'")

# # Clean up the temporary column
# df_gold.drop(columns=['current_target'], inplace=True)

# print("\n--- All Teacher Models Trained Successfully ---")

--- Loading Gold Standard Annotated Data from '/Users/zimoyang/Documents/swine_project/data/training_set_processed.csv' ---
Loaded 2000 gold-standard records.

--- Training Binary Teacher Model for: [01] Certain infectious or parasitic diseases (Index: 0) ---
Created a balanced training set with 494 records.
Saved model and vectorizer for '[01] Certain infectious or parasitic diseases'

--- Training Binary Teacher Model for: [08] Diseases of the nervous system (Index: 1) ---
Created a balanced training set with 80 records.
Saved model and vectorizer for '[08] Diseases of the nervous system'

--- Training Binary Teacher Model for: [12] Diseases of the respiratory system (Index: 2) ---
Created a balanced training set with 386 records.
Saved model and vectorizer for '[12] Diseases of the respiratory system'

--- Training Binary Teacher Model for: [13] Diseases of the digestive system (Index: 3) ---
Created a balanced training set with 458 records.
Saved model and vectorizer for '[13] Dise

## 2. Train and calibrate 11 teacher models
1. Load & Split Data: The script starts by loading the 2,000 expert-annotated records. It then splits this data into a 75% set for training and a 25% set for calibration.

2. Train a Specialist Model (for each label): The script loops through each of the 11 labels. In each loop, it trains a simple model on a special balanced dataset (created using downsampling). This forces the model to learn the patterns of even the rare diseases.

3. Calibrate the Model: After a model is trained, its sense of confidence is skewed because it learned from balanced data. The script then uses the held-out, imbalanced calibration set to adjust the model's confidence, making its probability scores more realistic and reliable.

4. Save the Final Models: Finally, the script saves each of the 11 fully trained and calibrated "teacher" models, along with their specific text vectorizers, to a new folder so they can be used in the next step: pseudo-labeling.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import re
import ast
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV # Import the calibrator
from sklearn.utils import resample

# --- 1. Configuration ---
ANNOTATED_DATA_FILE = '/Users/zimoyang/Documents/swine_project/data/training_set_processed.csv'
TEXT_COLUMN = 'anonymized_text_for_ml' # Use your preprocessed/anonymized text column
LABEL_VECTOR_COLUMN = 'expert_labels_vector'
RANDOM_STATE = 42
MODEL_OUTPUT_DIR = '/Users/zimoyang/Documents/swine_project/teacher_models_calibrated'

FINAL_MODELING_LABELS = [
    '[01] Certain infectious or parasitic diseases',
    '[08] Diseases of the nervous system',
    '[12] Diseases of the respiratory system',
    '[13] Diseases of the digestive system',
    '[14] Diseases of the skin',
    '[15] Diseases of the musculoskeletal system or connective tissue',
    '[18] Pregnancy, childbirth or the puerperium',
    '[19] Certain conditions originating in the perinatal period',
    'Monitoring',
    'Unknown',
    'Symptoms not classified elsewhere'
]

# --- 2. Load and Prepare Data ---
df_master = pd.read_csv(ANNOTATED_DATA_FILE)

# define df_gold as the 2000 annoatation data set with valid LABEL_VECTOR_COLUMN
df_gold = df_master.dropna(subset=[LABEL_VECTOR_COLUMN]).copy()
df_gold[LABEL_VECTOR_COLUMN] = df_gold[LABEL_VECTOR_COLUMN].apply(ast.literal_eval)
df_gold[TEXT_COLUMN] = df_gold[TEXT_COLUMN].astype(str)

# --- Create a Train/Calibration Split ---
# We'll use 75% of the data to train the base models and 25% to calibrate them.
df_train, df_calib = train_test_split(
    df_gold,
    test_size=0.25,
    random_state=RANDOM_STATE
)
print(f"Using {len(df_train)} records for training and {len(df_calib)} for calibration.")

os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# --- 3. Loop Through Each Label to Train and Calibrate ---
for label_index, label_name in enumerate(FINAL_MODELING_LABELS):
    print("\n" + "="*80)
    print(f"--- Processing: {label_name} ---")

    # a. Create a balanced training set from the main training data
    df_train['current_target'] = [vec[label_index] for vec in df_train[LABEL_VECTOR_COLUMN]]

    positive_class = df_train[df_train['current_target'] == 1]
    negative_class = df_train[df_train['current_target'] == 0]

    if len(positive_class) == 0:
        print(f"Skipping '{label_name}' as there are no positive examples in the training split.")
        continue

    negative_downsampled = resample(negative_class, n_samples=len(positive_class), random_state=RANDOM_STATE)
    balanced_training_data = pd.concat([positive_class, negative_downsampled])
    X_train_balanced_text = balanced_training_data[TEXT_COLUMN]
    y_train_balanced = balanced_training_data['current_target']

    # b. Train the base model on the balanced data
    vectorizer = TfidfVectorizer(max_features=3000, stop_words='english')
    X_train_balanced_tfidf = vectorizer.fit_transform(X_train_balanced_text)

    base_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
    base_model.fit(X_train_balanced_tfidf, y_train_balanced)

    # --- c. Calibrate the model ---
    print(f"Calibrating model for '{label_name}'...")
    # Prepare the held-out, imbalanced calibration data
    X_calib_text = df_calib[TEXT_COLUMN]
    y_calib_binary = np.array([vec[label_index] for vec in df_calib[LABEL_VECTOR_COLUMN]])

    # Transform the text using the same vectorizer
    X_calib_tfidf = vectorizer.transform(X_calib_text)

    # Wrap the base model in the calibrator and fit it on the calibration set
    # 'prefit' tells the calibrator that the model is already trained.
    calibrated_model = CalibratedClassifierCV(base_model, method='sigmoid', cv='prefit')
    calibrated_model.fit(X_calib_tfidf, y_calib_binary)

    # d. Save the FINAL CALIBRATED model and its vectorizer
    safe_label_name = re.sub(r'[\W_]+', '', label_name)
    model_path = os.path.join(MODEL_OUTPUT_DIR, f'model_{safe_label_name}.joblib')
    vectorizer_path = os.path.join(MODEL_OUTPUT_DIR, f'vectorizer_{safe_label_name}.joblib')

    joblib.dump(calibrated_model, model_path)
    joblib.dump(vectorizer, vectorizer_path)
    print(f"Saved CALIBRATED model and vectorizer for '{label_name}'")

print("\n--- All Teacher Models Trained and Calibrated Successfully ---")

Using 1500 records for training and 500 for calibration.

--- Processing: [01] Certain infectious or parasitic diseases ---
Calibrating model for '[01] Certain infectious or parasitic diseases'...
Saved CALIBRATED model and vectorizer for '[01] Certain infectious or parasitic diseases'

--- Processing: [08] Diseases of the nervous system ---
Calibrating model for '[08] Diseases of the nervous system'...
Saved CALIBRATED model and vectorizer for '[08] Diseases of the nervous system'

--- Processing: [12] Diseases of the respiratory system ---
Calibrating model for '[12] Diseases of the respiratory system'...
Saved CALIBRATED model and vectorizer for '[12] Diseases of the respiratory system'

--- Processing: [13] Diseases of the digestive system ---
Calibrating model for '[13] Diseases of the digestive system'...
Saved CALIBRATED model and vectorizer for '[13] Diseases of the digestive system'

--- Processing: [14] Diseases of the skin ---
Calibrating model for '[14] Diseases of the skin

/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/ana

Saved CALIBRATED model and vectorizer for '[15] Diseases of the musculoskeletal system or connective tissue'

--- Processing: [18] Pregnancy, childbirth or the puerperium ---
Calibrating model for '[18] Pregnancy, childbirth or the puerperium'...
Saved CALIBRATED model and vectorizer for '[18] Pregnancy, childbirth or the puerperium'

--- Processing: [19] Certain conditions originating in the perinatal period ---
Calibrating model for '[19] Certain conditions originating in the perinatal period'...
Saved CALIBRATED model and vectorizer for '[19] Certain conditions originating in the perinatal period'

--- Processing: Monitoring ---
Calibrating model for 'Monitoring'...
Saved CALIBRATED model and vectorizer for 'Monitoring'

--- Processing: Unknown ---
Calibrating model for 'Unknown'...
Saved CALIBRATED model and vectorizer for 'Unknown'

--- Processing: Symptoms not classified elsewhere ---
Calibrating model for 'Symptoms not classified elsewhere'...


/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/opt/ana

Saved CALIBRATED model and vectorizer for 'Symptoms not classified elsewhere'

--- All Teacher Models Trained and Calibrated Successfully ---
